Code largely based on the reaserach by the team of IBM's Qiskit.

More about the orignis of the coe can be found on
https://qiskit-community.github.io/qiskit-machine-learning/tutorials/07_pegasos_qsvc.html

### Installs

In [1]:
import pandas as pd

In [2]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 4.7 MB/s eta 0:00:00


In [3]:
pip install qiskit_machine_learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 7.9 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-py3-none-any.whl size=3566 sha256=32b0acc99d0f9fda22e2215d86547ce5f4305e18efa6cd8ab03d2bd0125db963
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


### Data Loading - Preprocessing

In [4]:
# Install the Kaggle library
!pip install kaggle

# Upload kaggle.json file
from google.colab import files
files.upload()  # This will prompt you to upload kaggle.json

# Make a directory for Kaggle and move the file there
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Now you can download the dataset
!kaggle datasets download -d ealaxi/paysim1 -p /content/datasets --unzip

print("Dataset downloaded to: /content/datasets")

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/ealaxi/paysim1
License(s): CC-BY-SA-4.0
 91% 162M/178M [00:01<00:00, 158MB/s]
100% 178M/178M [00:01<00:00, 137MB/s]
Dataset downloaded to: /content/datasets


In [5]:
# full_df=pd.read_csv("/content/datasets/PS_20174392719_1491204439457_log.csv",delimiter=',')


nRowsRead = 1000000
big_df = pd.read_csv("/content/datasets/PS_20174392719_1491204439457_log.csv",delimiter=',',nrows=nRowsRead)


nRowsRead = 10000
# Load the dataset (replace with actual filename if needed)
df = pd.read_csv("/content/datasets/PS_20174392719_1491204439457_log.csv",delimiter=',',nrows=nRowsRead)
print(df.head())

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


### Functions for Dataset

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

def prepare_data_for_training(df, target_column='isFraud', sampling_ratio=90/10, top_features=None, test_size=0.2, random_state=42):
    # Remove rows where 'isFraud' is empty
    df = df.dropna(subset=[target_column])

    print("Dataset shape after dropping empty 'isFraud' values:", df.shape)

    # Separate fraud and non-fraud transactions
    fraud_df = df[df[target_column] == 1]
    non_fraud_df = df[df[target_column] == 0]

    # Check the size of fraud and non-fraud subsets
    print("Size of fraud and non-fraud datasets:")
    print("Fraud:", fraud_df.shape, "Length of fraud_df:", len(fraud_df))
    print("Non-Fraud:", non_fraud_df.shape, "Length of non_fraud_df:", len(non_fraud_df))

    # Calculate the number of non-fraud samples needed for the specified ratio
    desired_non_fraud_count = int(len(fraud_df) * sampling_ratio)

    # Randomly sample the non-fraud data to the desired count
    non_fraud_sampled = non_fraud_df.sample(n=desired_non_fraud_count, random_state=random_state)

    # Concatenate fraud and sampled non-fraud data and shuffle
    balanced_df = pd.concat([fraud_df, non_fraud_sampled])
    balanced_df = balanced_df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    print("The balanced dataset now has shape of:", balanced_df.shape)
    print("The amount of Fraud Transactions in the Balanced Dataset is:", balanced_df[balanced_df[target_column] == 1].shape)

    # Print the class balance in the dataset
    print("Class balance after sampling:")
    print(balanced_df[target_column].value_counts(normalize=True) * 100)

    # One-hot encode categorical columns
    X = pd.get_dummies(balanced_df, columns=['type'])

    # Remove columns containing 'name'
    X = X.loc[:, ~X.columns.str.contains('name')]

    # Check for NaNs and handle them if present
    X = X.fillna(0).astype(int)

    # Separate target and features
    y = X[target_column]
    X = X.drop(columns=[target_column])

    # Feature selection using RandomForest
    model = RandomForestClassifier(random_state=random_state)
    model.fit(X, y)

    importances = model.feature_importances_
    feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': importances})
    feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

    print("Top 10 features based on importance:")
    print(feature_importance_df.head(10))

    # Select top features if provided; otherwise, use default top 5
    if top_features is None:
        top_features = feature_importance_df['feature'].head(4).tolist()

    X_top = X[top_features]

    # Scale the selected features
    scaler = StandardScaler()
    X_top = scaler.fit_transform(X_top)  # X_top is now a NumPy array

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_top, y, test_size=test_size, random_state=random_state
    )

    # Reset indices of y_train and y_test
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    # Convert y_train and y_test to NumPy arrays
    y_train = y_train.to_numpy()
    y_test = y_test.to_numpy()

    # Check for NaN values in training and testing sets
    print("NaN values in X_train:", np.isnan(X_train).sum())
    print("NaN values in X_test:", np.isnan(X_test).sum())
    print("NaN values in y_train:", np.isnan(y_train).sum())
    print("NaN values in y_test:", np.isnan(y_test).sum())

    # Print the sizes of the train and test sets
    print("Training set size:", X_train.shape, y_train.shape)
    print("Testing set size:", X_test.shape, y_test.shape)

    return X_train, X_test, y_train, y_test


In [7]:
X_big_train_top, X_big_test_top, y_big_train_top, y_big_test_top = prepare_data_for_training(big_df)

Dataset shape after dropping empty 'isFraud' values: (1000000, 11)
Size of fraud and non-fraud datasets:
Fraud: (535, 11) Length of fraud_df: 535
Non-Fraud: (999465, 11) Length of non_fraud_df: 999465
The balanced dataset now has shape of: (5350, 11)
The amount of Fraud Transactions in the Balanced Dataset is: (535, 11)
Class balance after sampling:
isFraud
0    90.0
1    10.0
Name: proportion, dtype: float64
Top 10 features based on importance:
           feature  importance
2    oldbalanceOrg    0.215188
1           amount    0.202810
5   newbalanceDest    0.147446
4   oldbalanceDest    0.103828
0             step    0.091704
11   type_TRANSFER    0.090694
3   newbalanceOrig    0.074054
10    type_PAYMENT    0.032093
8    type_CASH_OUT    0.022560
7     type_CASH_IN    0.019282
NaN values in X_train: 0
NaN values in X_test: 0
NaN values in y_train: 0
NaN values in y_test: 0
Training set size: (4280, 4) (4280,)
Testing set size: (1070, 4) (1070,)


In [8]:
X_train, X_test, y_train, y_test = prepare_data_for_training(df)

Dataset shape after dropping empty 'isFraud' values: (10000, 11)
Size of fraud and non-fraud datasets:
Fraud: (68, 11) Length of fraud_df: 68
Non-Fraud: (9932, 11) Length of non_fraud_df: 9932
The balanced dataset now has shape of: (680, 11)
The amount of Fraud Transactions in the Balanced Dataset is: (68, 11)
Class balance after sampling:
isFraud
0    90.0
1    10.0
Name: proportion, dtype: float64
Top 10 features based on importance:
           feature  importance
4   oldbalanceDest    0.200755
2    oldbalanceOrg    0.165474
5   newbalanceDest    0.137789
1           amount    0.120471
3   newbalanceOrig    0.090568
10    type_PAYMENT    0.083427
11   type_TRANSFER    0.076490
8    type_CASH_OUT    0.062391
0             step    0.044693
7     type_CASH_IN    0.013572
NaN values in X_train: 0
NaN values in X_test: 0
NaN values in y_train: 0
NaN values in y_test: 0
Training set size: (544, 4) (544,)
Testing set size: (136, 4) (136,)


In [9]:
pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 5.0 MB/s eta 0:00:00


In [10]:
from imblearn.over_sampling import SMOTE
import pandas as pd

from imblearn.over_sampling import SMOTE
import pandas as pd

def balance_with_smote(df, target_col='isFraud', smote_ratio=0.2, random_state=42):
    """Apply SMOTE to increase the minority class size by a specified ratio."""
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # Apply SMOTE with a limited ratio
    smote = SMOTE(sampling_strategy=smote_ratio, random_state=random_state)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    balanced_df = pd.DataFrame(X_resampled, columns=X.columns)
    balanced_df[target_col] = y_resampled
    return balanced_df

In [11]:
def count_fraud_percentage(df, fraud_col='isFraud'):
    """
    Calculate the count and percentage of fraud cases in a DataFrame.

    """

    fraud_count = df[fraud_col].sum()

    total_count = len(df)
    fraud_percentage = (fraud_count / total_count) * 100

    return fraud_count, fraud_percentage

### Pegasos Quantum Support Vector Classifier

#### Example from the Qiskit Implementation

In [ ]:
from sklearn.datasets import make_blobs

# example dataset
features, labels = make_blobs(n_samples=2000, n_features=2, centers=2, random_state=3, shuffle=True)

In [ ]:
features

array([[ 1.15397271,  5.10245931],
       [-3.83829162, -0.14283146],
       [ 0.7370821 ,  5.14172857],
       ...,
       [ 0.49298642,  4.09938337],
       [-3.84755613, -0.19948633],
       [ 0.45010503,  5.09191242]])

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(features)

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size=15, shuffle=False
)

In [ ]:
features

array([[2.36441004, 2.36625056],
       [0.91336337, 0.8943141 ],
       [2.24323702, 2.37727032],
       ...,
       [2.17228841, 2.08476682],
       [0.91067056, 0.87841558],
       [2.15982455, 2.36329088]])

In [ ]:
# number of qubits is equal to the number of features
num_qubits = 2

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000

In [ ]:
from qiskit.circuit.library import ZFeatureMap
from qiskit_algorithms.utils import algorithm_globals

from qiskit_machine_learning.kernels import FidelityQuantumKernel

algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)

qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [ ]:
from qiskit_machine_learning.algorithms import PegasosQSVC

pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

# training
pegasos_qsvc.fit(train_features, train_labels)

# testing
pegasos_score = pegasos_qsvc.score(test_features, test_labels)
print(f"PegasosQSVC classification test score: {pegasos_score}")

PegasosQSVC classification test score: 0.9945945945945946


#### Now Pegasos For Our Dataset

In [ ]:
df_cleaned = big_df.loc[(big_df['type'].isin(['CASH_OUT', 'TRANSFER'])),:]

In [ ]:
df_cleaned.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.0,0.0,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.0,C38997010,21182.0,0.00,1,0
15,1,CASH_OUT,229133.94,C905080434,15325.0,0.0,C476402209,5083.0,51513.44,0,0
19,1,TRANSFER,215310.30,C1670993182,705.0,0.0,C1100439041,22425.0,0.00,0,0
24,1,TRANSFER,311685.89,C1984094095,10835.0,0.0,C932583850,6267.0,2719172.89,0,0


In [ ]:
df_cleaned.drop(columns=['nameOrig','nameDest','isFlaggedFraud'],inplace=True)

<ipython-input-16-c11001ea31b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop(columns=['nameOrig','nameDest','isFlaggedFraud'],inplace=True)


In [ ]:
df_cleaned['type'] = big_df['type'].map({'CASH_OUT': 0, 'TRANSFER': 1})

<ipython-input-18-3b145fbfdf12>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['type'] = big_df['type'].map({'CASH_OUT': 0, 'TRANSFER': 1})


In [ ]:
df_smote_balanced = balance_with_smote(df_cleaned, target_col='isFraud', smote_ratio=0.2, random_state=42)
fraud_count_smote, fraud_percentage_smote = count_fraud_percentage(df_smote_balanced)
print(f"After SMOTE - Fraudulent: {fraud_count_smote}, Fraud Percentage: {fraud_percentage_smote:.2f}%")

After SMOTE - Fraudulent: 88913, Fraud Percentage: 16.67%


In [ ]:
final_df=df_smote_balanced

In [ ]:
final_df = final_df.sort_values(by='step')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
target_sample_size = len(final_df) // 1000
final_df_reduced, _ = train_test_split(final_df, train_size=target_sample_size, stratify=final_df['isFraud'])

In [ ]:
final_df_reduced=final_df_reduced.sort_values(by='step')

In [ ]:
# Separate features and target
X = final_df_reduced.drop(columns=['isFraud'])  # Replace 'target_column' with your target variable
y = final_df_reduced['isFraud']

# Split into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
num_qubits=X_train.shape[1]
tau = 100
C = 0.5

print(num_qubits,tau,C)

7 100 0.5


In [ ]:
pip install qiskit_algorithms

In [ ]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit_algorithms.utils import algorithm_globals

from qiskit_machine_learning.kernels import FidelityQuantumKernel


algorithm_globals.random_seed = 12345

feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=1)

qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [ ]:
X_test.shape

(107, 7)

In [ ]:
X_train.shape

(426, 7)

In [ ]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [ ]:
# Check for NaN values
print(X_train.isnull().sum())
print(y_train.isnull().sum())

# Optionally, drop NaN rows
X_train = X_train.dropna()
y_train = y_train.dropna()

step              0
type              0
amount            0
oldbalanceOrg     0
newbalanceOrig    0
oldbalanceDest    0
newbalanceDest    0
dtype: int64
0


In [ ]:
import numpy as np

# Convert DataFrames to numpy arrays
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

# Similarly for test set if needed
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()


In [ ]:
from qiskit_machine_learning.algorithms import PegasosQSVC

pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

# training
pegasos_qsvc.fit(X_train, y_train)

# testing
pegasos_score = pegasos_qsvc.score(X_test,y_test)
print(f"PegasosQSVC classification test score: {pegasos_score}")

PegasosQSVC classification test score: 0.8317757009345794


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Step 1: Make predictions on the test set
y_pred = pegasos_qsvc.predict(X_test)

# Step 2: Compute the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Step 3: Compute classification metrics
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.4f}")

# Recall
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.4f}")

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

# Step 4: Full classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)
